In [1]:
import socket
import pyarrow as pa

HOST = '127.0.0.1'
PORT = 9143

In [2]:
# read data from socket
def fetch_data():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        s.sendall(b'hello world\n')
        data = b''
        while True:
            newdata = s.recv(1024)
            if not newdata:
                break
            data += newdata
            print(f'received data len {len(newdata)}')
    return data

In [22]:
buffer = fetch_data()
print(f'total bytes received: {len(buffer)}')
reader = pa.ipc.open_stream(buffer)
table = reader.read_pandas()
print(table)
# table.to_pandas()

received data len 1024
received data len 1024
received data len 976
total bytes received: 3024


OSError: buffer_index out of range.

In [23]:
reader.stats

ReadStats(num_messages=2, num_record_batches=1, num_dictionary_batches=0, num_dictionary_deltas=0, num_replaced_dictionaries=0)

In [18]:
table = reader.read_all()


In [19]:
table

pyarrow.Table
partition key: string
bigintval: int64
blobval: binary
booleanval: bool
dateval: date32[day]
decimalval: struct<scale: int32, val: binary>
  child 0, scale: int32
  child 1, val: binary
doubleval: double
durationval: list<item: int64>
  child 0, item: int64
floatval: float
inetval: dense_union<ipv4: int32=0, ipv6: int64=1>
  child 0, ipv4: int32
  child 1, ipv6: int64
intval: int32
smallintval: int16
textval: string
timestampval: timestamp[ms]
timeuuidval: fixed_size_binary[16]
timeval: time64[ns]
tinyintval: int8
uuidval: fixed_size_binary[16]
varcharval: string
varintval: int64

In [10]:
gdf = cudf.DataFrame.from_arrow(table)
gdf

NameError: name 'cudf' is not defined

In [ ]:
bc = BlazingContext()
bc.create_table("classes", gdf)
bc.describe_table("classes")
result = bc.sql("SELECT * FROM classes")
result

In [ ]:
n_neighbors = 15

# we only take the first two features. We could avoid this ugly
# slicing by using a two-dim dataset
X = gdf.iloc[:, :2]
y = gdf.iloc[:, 2]

h = .02  # step size in the mesh

# Create color maps
cmap_light = ListedColormap(['orange', 'cyan', 'cornflowerblue'])
cmap_bold = ['darkorange', 'c', 'darkblue']

weights = 'uniform'

# we create an instance of Neighbours Classifier and fit the data.
clf = KNeighborsClassifier(n_neighbors=15)
clf.fit(X, y)

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X.iloc[:, 0].min() - 1, X.iloc[:, 0].max() + 1
y_min, y_max = X.iloc[:, 1].min() - 1, X.iloc[:, 1].max() + 1
xx, yy = cupy.meshgrid(cupy.arange(x_min, x_max, h),
                        cupy.arange(y_min, y_max, h))
Z = clf.predict(cupy.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(8, 6))
plt.contourf(xx.get(), yy.get(), Z.get(), cmap=cmap_light)

# Plot also the training points
sns.scatterplot(x=X.iloc[:, 0].to_array(), y=X.iloc[:, 1].to_array(), hue=np.array(['setosa', 'versicolor', 'virginica'])[y.to_array()],
                palette=cmap_bold, alpha=1.0, edgecolor="black")
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("3-Class classification (k = %i, weights = '%s')"
            % (n_neighbors, weights))

plt.xlabel('sepal length (cm)')
plt.ylabel('sepal width (cm)')

plt.show()